In [1]:
import numpy as np
from surrogate_models import surrogate_models
import pickle
from scipy.stats import norm

# Importing Simulator

In [2]:
import LStates.gfun_63 as gfun_63  # function = gfun_63.gfun_63  #assigning ground truth sim (it should return 1dim only)

from LStates.HMCMC_functions import example_2, example_4

#example_2 Parabolic/Concave limit-state function
#ref. (HMCMC High Dim, Prof. Kostas) PF= 5.90E-8
# 2 independent standard normal random variables

#example_4: The Himmelblau Function
#ref. (HMCMC High Dim, Prof. Kostas)
# 2 independent standard normal random variables

from LStates.g11D_electric import example_electric

Marginals for g11D_electric function

In [3]:
# X1 = L (m) mean=4.2    cov=0.10   lognormal/R>0
x1_mean = 4.2
x1_std = x1_mean * 0.1
normal_std_1 = np.sqrt(np.log(1 + (x1_std/x1_mean)**2))
normal_mean_1 = np.log(x1_mean) - normal_std_1**2 / 2

# X2 = h (m) mean=0.02    cov=0.10 lognormal /R>0
x2_mean = 0.02
x2_std = x2_mean * 0.1
normal_std_2 = np.sqrt(np.log(1 + (x2_std/x2_mean)**2))
normal_mean_2 = np.log(x2_mean) - normal_std_2**2 / 2

# X3 = d (m) mean=0.001  cov=0.05 lognormal /R>0
x3_mean = 0.001
x3_std = x3_mean * 0.05
normal_std_3 = np.sqrt(np.log(1 + (x3_std/x3_mean)**2))
normal_mean_3 = np.log(x3_mean) - normal_std_3**2 / 2

# X4 = ZL () mean=1000    cov=0.20 lognormal /R>0
x4_mean = 1000
x4_std = x4_mean * 0.2
normal_std_4 = np.sqrt(np.log(1 + (x4_std/x4_mean)**2))
normal_mean_4 = np.log(x4_mean) - normal_std_4**2 / 2

# X5 = Z0 () mean=50   cov=0.05 lognormal /R>0
x5_mean = 50
x5_std = x5_mean * 0.05
normal_std_5 = np.sqrt(np.log(1 + (x5_std/x5_mean)**2))
normal_mean_5 = np.log(x5_mean) - normal_std_5**2 / 2

# X6 = ae (V/m) mean=1  cov=0.20 lognormal /R>0
x6_mean = 1
x6_std = x6_mean * 0.2
normal_std_6 = np.sqrt(np.log(1 + (x6_std/x6_mean)**2))
normal_mean_6 = np.log(x6_mean) - normal_std_6**2 / 2

# X7 = theta_e (rad) mean=pi/4    cov=0.577 uniform / [0,pi/2]
x7_min = 0
x7_max = np.pi / 2

# X8 = theta_p (rad) mean=pi/4    cov=0.577 uniform / [0,pi/2]
x8_min = 0
x8_max = np.pi / 2

# X9 = phi_p (rad) mean=pi   cov=0.577 uniform / [0,pi*2]
x9_min = 0
x9_max = np.pi*2

# X10 = f (MHz) mean=30    cov=0.096 uniform / [25 ,35]
x10_min = 25.
x10_max = 35.

# X11 = alpha (-) mean=0.0010  cov=0.289 uniform / [0.0005 , 0.0015]
x11_min = 0.0005
x11_max =  0.0015

In [4]:
dim = 11
n_mcs = int(1e8)
X = np.zeros((n_mcs,dim))
L = X[:,0] = np.random.lognormal(mean=normal_mean_1, sigma=normal_std_1, size=n_mcs)   #L (m)
h = X[:,1] = np.random.lognormal(mean=normal_mean_2, sigma=normal_std_2, size=n_mcs)   #h (m)
d = X[:,2] = np.random.lognormal(mean=normal_mean_3, sigma=normal_std_3, size=n_mcs)   #d (m)
ZL = X[:,3] = np.random.lognormal(mean=normal_mean_4, sigma=normal_std_4, size=n_mcs)   #ZL ()
Z0 = X[:,4] = np.random.lognormal(mean=normal_mean_5, sigma=normal_std_5, size=n_mcs)   #Z0 ()
ae = X[:,5] = np.random.lognormal(mean=normal_mean_6, sigma=normal_std_6, size=n_mcs)   #ae (V/m)
theta_e = X[:,6] = np.random.uniform(low=x7_min, high=x7_max, size=n_mcs)        #theta_e (rad)
theta_p = X[:,7] = np.random.uniform(low=x8_min, high=x8_max, size=n_mcs)        #theta_p (rad)
phi_p =  X[:,8] = np.random.uniform(low=x9_min, high=x9_max, size=n_mcs)        #phi_p (rad)
f = X[:,9] = np.random.uniform(low=x10_min, high=x10_max, size=n_mcs)      #f (MHz)
alpha = X[:,10] = np.random.uniform(low=x11_min, high=x11_max, size=n_mcs)     #alpha (-)

In [5]:
g_test = example_electric(X)
np.sum(g_test < 0) / n_mcs

0.0

In [ ]:
function = example_4
dim = 2
n_mcs = int(1e6)
Xtest = np.random.normal(0.0, 1.0, size=(n_mcs, dim))
y_test = function(Xtest)

Pf_ref = np.sum( y_test < 0 ) / n_mcs
B_ref= - norm.ppf( Pf_ref )
Pf_ref, B_ref

# Setting up training conditions

In [ ]:
#Bulding desing of experiment for LHS
dim = 2
marginal = 'hermite'  #random normal
# marginal = 'legendre' #random uniform

exp = {}
for var in range(dim):
    var_name = 'x'+str(var+1)
    exp[var_name] = [0.0, 1.0, marginal]   #initial design domain for each variable (normal, uniform)
                                           # {'x1':[Min, Max, 'legendre'], 'x2':[mean, std, 'hermite']}

In [ ]:
n_mcs = int(3e6)      #number of MC samples
passive_samples = 10 
active_samples = 50
function = example_4

#to creat PCK model--------------------------------------------
settings_PCK = {"metamodel" : 'PCK',
                     "max_pol": 8,  #max polynomial degree
                     "passive_sampling" : [passive_samples,'LHS'] , # initial samplings [n_samples, method]
                     "marginals" : [exp],   # {'x1':[Min, Max, 'legendre'], 'x2':[mean, var, hermite]}
                     "active_sampling" : [function, 'U'] } #target settings [n_act, targetF, learningF(U, EFF, ULOO, EFFLOO)]                  

#to creat Kriging model--------------------------------------------
settings_GP = {"metamodel" : 'GP',
               "passive_sampling" : [passive_samples,'LHS'] , # initial samplings [n_samples, method]
               "marginals" : [exp],   # {'x1':[Min, Max], 'x2':[mean, var]}  ( if Legendre , if Hermite)
               "active_sampling" : [function, 'U']} #target settings [n_act, targetF, learningF(U, EFF, ULOO, EFFLOO)]                  

#creating objects
PCK_batch = surrogate_models(settings_PCK)
GP_batch = surrogate_models(settings_GP)

# Active Training

Kriging model with Matern Kernel (scikit learn)

In [ ]:
experiments = 5
GP_batch.active_sampling(n_exp = experiments, n_act = active_samples, MCpool= n_mcs, GroundT=False)  

PCK model

In [ ]:
experiments = 5
PCK_batch.active_sampling(n_exp = experiments, n_act = active_samples, MCpool= n_mcs, GroundT=True)  

Loading results file

In [ ]:
file_name = 'PCK_Batch_1.sav'
experiment = pickle.load(open(file_name, 'rb'))

In [ ]:
experiment['last']

In [ ]:
dim = 2
n_mcs = int(3e6) 

Xtest = np.random.normal(0.0, 1.0, size=(n_mcs, dim))
y_test = example_4(Xtest)

In [ ]:
pf = np.sum(y_test <0) / n_mcs
pf